In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 11.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.autograd import Variable

import torchvision
from torchvision import models
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, ToPILImage
from torchvision.io import read_image
from torchvision import transforms
from torchmetrics.classification import BinaryPrecision, BinaryRecall, BinaryF1Score
import statistics

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch.utils.data
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, Subset
from torch.utils.data import WeightedRandomSampler
from sklearn.utils.class_weight import compute_class_weight

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import copy
import multiprocessing as mp
from tqdm import tqdm

from collections import Counter

import statistics
import time

import glob

from torch.utils.data import DataLoader
from torchvision import models

from torchvision.models import resnet18, ResNet18_Weights

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

mp.set_start_method('spawn', force=True)

In [ ]:
im_path = '.../path/...'
im = Image.open(im_path)
image_array = np.array(im)
print(image_array.max())
print(image_array.min())
print(im.mode)
_, file_extension = os.path.splitext(im_path)
print(file_extension)

255
0
L
.png


In [ ]:
im_path = '.../path/...'
im = Image.open(im_path)
image_array = np.array(im)
print(image_array.max())
print(image_array.min())
print(im.mode)
_, file_extension = os.path.splitext(im_path)
print(file_extension)

255
0
L
.png


In [ ]:
class MyDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.labels = [0 if "Normal" in path else 1 for path in image_paths]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('L')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
normal_images_pattern = '.../path/...'
pneumonia_images_pattern = '.../path/...'

normal_image_paths = glob.glob(normal_images_pattern)
pneumonia_image_paths = glob.glob(pneumonia_images_pattern)

image_paths = normal_image_paths + pneumonia_image_paths

In [ ]:
transform = transforms.Compose([transforms.Resize((256, 256)),
                               transforms.ToTensor()])

In [ ]:
dataset = MyDataset(image_paths, transform=transform)

total_count = len(dataset)
train_count = int(0.7 * total_count)
valid_count = int(0.15 * total_count)
test_count = total_count - train_count - valid_count

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_count, valid_count, test_count])

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

print("Total images:", total_count)
print("Training set size:", len(train_dataset))
print("Validation set size:", len(valid_dataset))
print("Test set size:", len(test_dataset))

Total images: 11537
Training set size: 8075
Validation set size: 1730
Test set size: 1732


In [ ]:
count_label_0 = 0
count_label_1 = 0

for idx in train_dataset.indices:
    label = dataset.labels[idx]
    if label == 0:
        count_label_0 += 1
    elif label == 1:
        count_label_1 += 1

print(f"Count of Label 0 (Normal): {count_label_0}")
print(f"Count of Label 1 (Viral Pneumonia): {count_label_1}")

Count of Label 0 (Normal): 7132
Count of Label 1 (Viral Pneumonia): 943


In [ ]:
# Total samples
total_samples = count_label_0 + count_label_1

# Calculate weights for each class
weight_for_0 = total_samples / (2 * count_label_0)
weight_for_1 = total_samples / (2 * count_label_1)

# Create a tensor of weights for use with the loss function
class_weights = torch.tensor([weight_for_0, weight_for_1], dtype=torch.float32)

In [ ]:
class_weights

tensor([0.5675, 4.2014])

In [ ]:
original_label_0 = sum(1 for label in dataset.labels if label == 0)
original_label_1 = sum(1 for label in dataset.labels if label == 1)
print(f"Original Count of Label 0: {original_label_0}")
print(f"Original Count of Label 1: {original_label_1}")

Original Count of Label 0: 10192
Original Count of Label 1: 1345


In [ ]:
# CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET
# CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET
# CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET
# CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET
# CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET CREATE SUBSET

# Total indices and labels from the dataset
total_indices = list(range(len(dataset)))
total_labels = dataset.labels  # Ensure this attribute exists and correctly corresponds to the indices

# Step 1: Use train_test_split to create a stratified subset of 1000 samples
subset_indices, _ = train_test_split(total_indices, train_size=1000, stratify=total_labels, random_state=42)

# Step 2: Use train_test_split again to split the subset into train, validation, and test sets
train_indices, test_indices = train_test_split(subset_indices, test_size=0.3, stratify=[total_labels[i] for i in subset_indices], random_state=42)
valid_indices, test_indices = train_test_split(test_indices, test_size=0.5, stratify=[total_labels[i] for i in test_indices], random_state=42)

# Step 3: Create Subset instances for train, validation, and test sets
train_dataset = Subset(dataset, train_indices)
valid_dataset = Subset(dataset, valid_indices)
test_dataset = Subset(dataset, test_indices)

# Step 4: Create DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

# Print information about the splits
print("Total images in subset:", 1000)
print("Training set size:", len(train_dataset))
print("Validation set size:", len(valid_dataset))
print("Test set size:", len(test_dataset))

Total images in subset: 1000
Training set size: 700
Validation set size: 150
Test set size: 150


In [ ]:
count_label_0 = 0
count_label_1 = 0

for idx in train_dataset.indices:
    label = dataset.labels[idx]
    if label == 0:
        count_label_0 += 1
    elif label == 1:
        count_label_1 += 1

print(f"Count of Label 0 (Normal): {count_label_0}")
print(f"Count of Label 1 (Viral Pneumonia): {count_label_1}")

Count of Label 0 (Normal): 618
Count of Label 1 (Viral Pneumonia): 82


In [ ]:
total_samples = count_label_0 + count_label_1

weight_for_0 = total_samples / (2 * count_label_0)
weight_for_1 = total_samples / (2 * count_label_1)

class_weights = torch.tensor([weight_for_0, weight_for_1], dtype=torch.float32)

In [ ]:
class_weights

tensor([0.5663, 4.2683])

In [ ]:
transform = Compose([
    Resize((256, 256)),
    ToTensor(),
    Normalize(mean=[0.485], std=[0.229])])

In [ ]:
lr_choices = [0.001, 0.01, 0.05, 0.1]
batch_size_choices = [8, 16, 32, 64]
optimizer_choices = [optim.Adam, optim.SGD]

num_search_iterations = 5

In [ ]:
def create_model(lr, optimizer_cls):
    model = resnet18(weights=ResNet18_Weights.DEFAULT)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    optimizer = optimizer_cls(model.parameters(), lr=lr)
    return model, optimizer

In [ ]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10, patience=5):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        true_labels = []
        pred_labels = []

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.tolist())
            pred_labels.extend(predicted.tolist())

        train_accuracy = accuracy_score(true_labels, pred_labels)
        train_precision = precision_score(true_labels, pred_labels, zero_division=0)
        train_recall = recall_score(true_labels, pred_labels)
        train_f1 = f1_score(true_labels, pred_labels)

        model.eval()
        val_loss = 0
        val_true_labels = []
        val_pred_labels = []

        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                val_true_labels.extend(labels.tolist())
                val_pred_labels.extend(predicted.tolist())

        val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
        val_precision = precision_score(val_true_labels, val_pred_labels, zero_division=0)
        val_recall = recall_score(val_true_labels, val_pred_labels)
        val_f1 = f1_score(val_true_labels, val_pred_labels)

        avg_train_loss = total_loss / len(train_loader)
        avg_val_loss = val_loss / len(valid_loader)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model = model
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print('Early stopping!')
                break

        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')
        print(f'Validation - Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}')

    return best_model

In [ ]:
for i in range(num_search_iterations):
    lr = np.random.choice(lr_choices)
    batch_size = np.random.choice(batch_size_choices)
    optimizer_cls = np.random.choice(optimizer_choices)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

    model, optimizer = create_model(lr, optimizer_cls)
    criterion = nn.CrossEntropyLoss()

    print(f"Starting Trial {i+1}: Learning Rate = {lr}, Batch Size = {batch_size}, Optimizer = {optimizer_cls.__name__}")
    trained_model = train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=6)

Starting Trial 1: Learning Rate = 0.1, Batch Size = 16, Optimizer = Adam
Epoch 1, Train Loss: 1.5231, Val Loss: 957.4786
Train - Accuracy: 0.8643, Precision: 0.0667, Recall: 0.0122, F1: 0.0206
Validation - Accuracy: 0.8800, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Epoch 2, Train Loss: 0.2602, Val Loss: 0.5198
Train - Accuracy: 0.8843, Precision: 0.6667, Recall: 0.0244, F1: 0.0471
Validation - Accuracy: 0.8800, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Epoch 3, Train Loss: 0.2190, Val Loss: 0.1863
Train - Accuracy: 0.8943, Precision: 0.5667, Recall: 0.4146, F1: 0.4789
Validation - Accuracy: 0.8867, Precision: 0.5185, Recall: 0.7778, F1: 0.6222
Epoch 4, Train Loss: 0.2013, Val Loss: 0.4935
Train - Accuracy: 0.8914, Precision: 0.5469, Recall: 0.4268, F1: 0.4795
Validation - Accuracy: 0.6667, Precision: 0.2647, Recall: 1.0000, F1: 0.4186
Epoch 5, Train Loss: 0.2220, Val Loss: 0.3075
Train - Accuracy: 0.9000, Precision: 0.5882, Recall: 0.4878, F1: 0.5333
Validation - Accuracy: 0.84

In [ ]:
# TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET
# TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET
# TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET
# TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET
# TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET TRAIN MODEL ON THE DATASET

def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10, patience=5):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        true_labels = []
        pred_labels = []
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.tolist())
            pred_labels.extend(predicted.tolist())

        train_accuracy = accuracy_score(true_labels, pred_labels)
        train_precision = precision_score(true_labels, pred_labels, zero_division=0)
        train_recall = recall_score(true_labels, pred_labels)
        train_f1 = f1_score(true_labels, pred_labels)

        model.eval()
        val_loss = 0
        val_true_labels = []
        val_pred_labels = []
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_true_labels.extend(labels.tolist())
                val_pred_labels.extend(predicted.tolist())

        val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
        val_precision = precision_score(val_true_labels, val_pred_labels, zero_division=0)
        val_recall = recall_score(val_true_labels, val_pred_labels)
        val_f1 = f1_score(val_true_labels, val_pred_labels)

        avg_train_loss = total_loss / len(train_loader)
        avg_val_loss = val_loss / len(valid_loader)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
            print('Model saved as validation loss improved.')
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print('Early stopping!')
                break

        print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        print(f'Train - Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')
        print(f'Validation - Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}')

    model.load_state_dict(torch.load('best_model.pth'))  # Load the best model found during training
    return model

In [ ]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 2)

optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss(class_weights)

trained_model = train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=10, patience=5)


Starting epoch 1
Epoch 1: Train Loss: 0.4892, Val Loss: 0.4725
Train - Accuracy: 0.805, Precision: 0.812, Recall: 0.798, F1: 0.805
Validation - Accuracy: 0.810, Precision: 0.817, Recall: 0.803, F1: 0.810

Starting epoch 2
Epoch 2: Train Loss: 0.3784, Val Loss: 0.3617
Train - Accuracy: 0.857, Precision: 0.865, Recall: 0.851, F1: 0.858
Validation - Accuracy: 0.862, Precision: 0.870, Recall: 0.856, F1: 0.863

Starting epoch 3
Epoch 3: Train Loss: 0.3123, Val Loss: 0.3051
Train - Accuracy: 0.887, Precision: 0.895, Recall: 0.880, F1: 0.887
Validation - Accuracy: 0.892, Precision: 0.900, Recall: 0.885, F1: 0.892

Starting epoch 4
Epoch 4: Train Loss: 0.2761, Val Loss: 0.2685
Train - Accuracy: 0.905, Precision: 0.913, Recall: 0.899, F1: 0.906
Validation - Accuracy: 0.910, Precision: 0.918, Recall: 0.903, F1: 0.910

Starting epoch 5
Epoch 5: Train Loss: 0.2498, Val Loss: 0.2430
Train - Accuracy: 0.915, Precision: 0.923, Recall: 0.909, F1: 0.916
Validation - Accuracy: 0.920, Precision: 0.928, 

In [ ]:
def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    test_true_labels = []
    test_pred_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            test_true_labels.extend(labels.tolist())
            test_pred_labels.extend(predicted.tolist())

    test_accuracy = accuracy_score(test_true_labels, test_pred_labels)
    test_precision = precision_score(test_true_labels, test_pred_labels, zero_division=0)
    test_recall = recall_score(test_true_labels, test_pred_labels)
    test_f1 = f1_score(test_true_labels, test_pred_labels)

    print(f'Test Loss: {test_loss / len(test_loader):.4f}')
    print(f'Test - Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')

evaluate_model(trained_model, test_loader, criterion)


Test Loss: 0.1798
Test - Accuracy: 0.952, Precision: 0.960, Recall: 0.948, F1: 0.954

